Run a complete test of the NearbyInfantReact process, which incorporates the HU_RAPID_SINGLE filter, a chained T2 catalog match and then either RapidBase or RapidLco as output.. 

In [ ]:
import os
%load_ext ampel_quick_import
%qi DevAmpelContext DictSecretProvider AmpelLogger T2Processor ChannelModel AlertConsumer TarAlertLoader ChannelModel AbsAlertFilter T3Processor TransientInfoPrinter

In [ ]:
# See other sample notebooks for path to download
#ALERT_ARCHIVE = "/home/jnordin/data/ztfalerts/ztf_public_20180731_nostars.tar.gz"
#ALERT_ARCHIVE = "/home/jnordin/github/ampelv07/Ampel-notebooks/ztf/skyportal/HU_TNS_PARTNER.tar.gz"
ALERT_ARCHIVE = "/home/jnordin/github/ampelv07/Ampel-notebooks/ztf/skyportal/HU_RAPID|HU_RAPID_SINGLE.tar.gz"
AMPEL_CONF = "/home/jnordin/github/ampelv71/ampel_conf.yaml"

In [ ]:
secrets = DictSecretProvider( {'slack/ztf_general/jno': None,
                              'mongo/logger':{},
                              'mongo/writer':{},
                              'extcats/reader':{},
                              'lco/jnordin':None,
                              })

In [ ]:
ctx = DevAmpelContext.load(
    config_file_path = AMPEL_CONF,
    db_prefix = "AmpelTestAP",
    purge_db = True,
    secrets = secrets,
)

In [ ]:
ctx.add_channel(
    name="TEST_CHAN2",
    access=['ZTF', 'ZTF_PUB', 'ZTF_COLLAB']
)

In [ ]:
cat_conf = { 'catalogs' : {
             'SDSS_spec' : {
                 'use' : 'extcats',
                 'catq_kwargs' : {
                     'ra_key' : 'ra',
                     'dec_key' : 'dec',
                 },
                 'rs_arcsec' : 10,
                 'keys_to_append' : ['z','bptclass','subclass'],
                 'pre_filter' : None,
                 'post_filter' : None,
             },
             'milliquas' : {
                 'use' : 'extcats',
                 'catq_kwargs' : {
                     'ra_key' : 'ra',
                     'dec_key' : 'dec',
                 },
                 'rs_arcsec' : 3,
                 'keys_to_append' : ['broad_type', 'name', 'redshift', 'qso_prob'],
                 'pre_filter' : None,
                 'post_filter' : None,
             }
        },
        'extcats' : {
            'auth' : {'key' : 'extcats/reader'},
            'max_time' : 300.0            
        }
}

In [ ]:
extcat_config_id = ctx.add_config_id( cat_conf )
print(extcat_config_id)

In [ ]:
point_conf = {
    "max_age": 500,
    "det_filterids" : [1,2],
    "t2_dependency": [
        {
            "unit": "T2CatalogMatch",
            "config": extcat_config_id,
            "link_override": {
                "pps": "last"
                }
        }
    ]
}

In [ ]:
point_config_id = ctx.add_config_id( point_conf )
print(point_config_id)

In [ ]:
ap = AlertConsumer(
    context = ctx,
    process_name = "ipyton_notebook_test",
    supplier = "ZiAlertSupplier",
    log_profile = "debug",
    directives = [
        {
            "channel": "TEST_CHAN2", 
            "filter": {"unit": "SimpleDecentFilter","config": {
                                                                "min_drb":0.995,
                                                                "min_ndet":1,
                                                                "min_tspan":0.0,
                                                                "max_tspan" : 3,
                                                                "min_gal_lat" : 15
                                                              }
                        },
            "stock_update": "ZiStockIngester",
            't0_add': {
                "ingester": "ZiAlertContentIngester",
                "t1_combine": [
                    {
                        "ingester": "PhotoCompoundIngester",
                        "config": {"combiner": "ZiT1Combiner"},
                        "t2_compute": {
                            "ingester": "PhotoT2Ingester",
                            "config": {"tags": ["ZTF"]},
                            "units": [
                                {'unit': 'T2InfantCatalogEval',
                                 'config': point_config_id
                                },
                            ]
                        }
                    }
                ],
                "t2_compute": {
                    'unit': 'DualPointT2Ingester',
                    'config': {'tags': ['ZTF']},
                    'units': [
                        {
                            'unit': 'T2CatalogMatch', 
                            'config' : extcat_config_id,
                            "ingest": {'eligible': {'pps': 'first'}}
                        }
                    ],
                },
            }
        }
    ]
)

In [ ]:
ap.set_loader(TarAlertLoader(ALERT_ARCHIVE))

In [ ]:
ap.set_iter_max(10000)

In [ ]:
ap.run()

In [ ]:
t2p = T2Processor(context=ctx, process_name="T2Processor_test", log_profile="debug")

In [ ]:
t2p.run()

In [ ]:
# Test base
t3p = T3Processor(
    context=ctx,
    process_name = "T3Processor_test",
    log_profile = "default", # debug
    channel = "TEST_CHAN2",
    directives = [ {
        "context": [
            {"unit": "T3AddLastRunTime"},
            {"unit": "T3AddAlertsNumber"}
        ],
        "select": {
            "unit": "T3FilteringStockSelector",
            "config": {
                "created": {"after": {"match_type": "time_delta", "days": -400000}},
                'tag': {'with': "ZTF", 'without': 'HAS_ERROR'},
                't2_filter':  {
                    'unit': 'T2InfantCatalogEval',
                    'match': {'action': True}
                }, 
            }
        },
        "load": {
            "unit": "T3SimpleDataLoader",
            "config": {
                "directives": ["TRANSIENT", "DATAPOINT", "COMPOUND", "T2RECORD"],
            }

        },
        "run": {
            "unit": "T3UnitRunner",
            "config": {
                "directives": [
                      {
                            "project": {
                                "unit": "T3ChannelProjector",
                                "config": {
                                    "channel": "TEST_CHAN2"
                                }
                            },
                            "execute": [
                                {"unit": "TransientInfoPrinter"},
                                {
                                    "unit": "RapidBase",
                                    "config": {
                                        'do_react': False,
                                        'do_testreact': True,
                                        'slack_token': {
                                            "key": "slack/ztf_general/jno",
                                        },
                                        'slack_channel': '#ztf_auto',
                                        'slack_username': 'AMPEL',
                                        't2info_from' : ['T2InfantCatalogEval']
                                    },
                                },
                            ]

                      }
                ]
            }
        }
    } ]
)


In [ ]:
# Test Lco
t3p2 = T3Processor(
    context=ctx,
    process_name = "T3Processor_test",
    log_profile = "default", # debug
    channel = "TEST_CHAN2",
    directives = [ {
        "context": [
            {"unit": "T3AddLastRunTime"},
            {"unit": "T3AddAlertsNumber"}
        ],
        "select": {
            "unit": "T3FilteringStockSelector",
            "config": {
                "created": {"after": {"match_type": "time_delta", "days": -400000}},
                'tag': {'with': "ZTF", 'without': 'HAS_ERROR'},
                't2_filter':  {
                    'unit': 'T2InfantCatalogEval',
                    'match': {'action': True}
                }, 
            }
        },
        "load": {
            "unit": "T3SimpleDataLoader",
            "config": {
                "directives": ["TRANSIENT", "DATAPOINT", "COMPOUND", "T2RECORD"],
            }

        },
        "run": {
            "unit": "T3UnitRunner",
            "config": {
                "directives": [
                      {
                            "project": {
                                "unit": "T3ChannelProjector",
                                "config": {
                                    "channel": "TEST_CHAN2"
                                }
                            },
                            "execute": [
                                {"unit": "TransientInfoPrinter"},
                                {
                                    "unit": "RapidLco",
                                    "config": {
                                        'do_react': True,
                                        'do_testreact': True,
                                        'slack_token': {
                                            "key": "slack/ztf_general/jno",
                                        },
                                        'slack_channel': '#ztf_auto',
                                        'slack_username': 'AMPEL',
                                        't2info_from' : ['T2InfantCatalogEval'],
                                        'lco_api': {
                                            "key": "lco/jnordin",
                                        }
                                    },
                                },
                            ]

                      }
                ]
            }
        }
    } ]
)


In [ ]:
t3p2.run()